# EDA Notebook

In this notebook, we will use a public revenue dataset obtained from [Kaggle](https://www.kaggle.com/datasets/census/business-and-industry-reports?select=notes.txt) and pre-process the dataset to be used for training an Auto ARIMA time series forecasting model to predict revenue in the future.

In [1]:
import pandas as pd
import os

## Helper Functions

In [2]:
# function to filter first n groups after grouping a df
def filter_n_groups(g_df, n):
    result = [g[1] for g in list(g_df)[:n]]
    df_result = pd.concat(result)
    return df_result

## Data Collection

In [3]:
# Read the Kaggle dataset
df = pd.read_csv("../data/raw/data.csv")

/tmp/ipykernel_4168/4251968774.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/data.csv")


In [4]:
df.head()

,time_series_code,date,value
0,BOPGS_BAL_US_adj,1992-01-01,-2026
1,BOPGS_EXP_US_adj,1992-01-01,50251
2,BOPGS_IMP_US_adj,1992-01-01,52277
3,BOPG_BAL_US,1992-01-01,-6210
4,BOPG_EXP_US,1992-01-01,33953


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333709 entries, 0 to 1333708
Data columns (total 3 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   time_series_code  1333709 non-null  object
 1   date              1333709 non-null  object
 2   value             1333709 non-null  object
dtypes: object(3)
memory usage: 30.5+ MB


In [6]:
# convert revenue values to numerical format
df['value'] = pd.to_numeric(df['value'], errors='coerce')

## Data Pre-Processing

We will now pre-process the raw data set into a suitable format for training our model.

In [7]:
# Only retain positive revenue values
df_new = df[df['value'] > 0]

In [8]:
# rename column
df_new = df_new.rename(columns={"time_series_code": "account_name"})

In [9]:
df_new.head()

,account_name,date,value
1,BOPGS_EXP_US_adj,1992-01-01,50251.0
2,BOPGS_IMP_US_adj,1992-01-01,52277.0
4,BOPG_EXP_US,1992-01-01,33953.0
5,BOPG_IMP_US,1992-01-01,40163.0
7,BOPGS_EXP_US_adj,1992-02-01,51682.0


In [10]:
# Let's filter for revenues between 2012-2014 to be used as our training set
# fist 3 months of 2015 will be used as our validation set
df_filtered = df_new[(df_new['date'] >= '2012-01-01') & (df_new['date'] <= '2015-03-01')]

In [11]:
df_filtered.head()

,account_name,date,value
1441,BOPGS_EXP_US_adj,2012-01-01,180073.0
1442,BOPGS_IMP_US_adj,2012-01-01,231056.0
1444,BOPG_EXP_US,2012-01-01,119238.0
1445,BOPG_IMP_US,2012-01-01,185993.0
1447,BOPGS_EXP_US_adj,2012-02-01,182640.0


In [12]:
df_filtered.tail()

,account_name,date,value
1325351,20IX_MPCV_US_adj,2015-03-01,2.1
1325352,XXXX_MPCP_US_adj,2015-03-01,1.6
1325353,00XX_MPCP_US_adj,2015-03-01,10.7
1325354,NRXX_MPCP_US_adj,2015-03-01,1.4
1325356,03XX_MPCP_US_adj,2015-03-01,5.2


In [13]:
len(df_filtered)

199269

Let's filter for only those accounts that have **all** the monthly revenue numbers present between 2012-01 to 2015-03 i.e. for 39 months.

In [14]:
grouped_df = df_filtered.groupby("account_name")

In [15]:
final_df = pd.DataFrame()
for group in grouped_df:
    if group[1]['date'].nunique() == 39:
        final_df = pd.concat([final_df, group[1]])
final_df.reset_index(inplace=True, drop=True)

In [16]:
final_df.account_name.nunique()

2045

Now that we have about ~2000 unique account names, let's filter to retain only a subset of the accounts for model training purposes.

In [17]:
g_df = final_df.groupby("account_name")

In [18]:
# filter for first 50 unique account names
df_result = filter_n_groups(g_df, 50)

In [19]:
df_result.head()

,account_name,date,value
0,00XX_E_MPCP_US,2012-01-01,3.2
1,00XX_E_MPCP_US,2012-02-01,3.8
2,00XX_E_MPCP_US,2012-03-01,2.5
3,00XX_E_MPCP_US,2012-04-01,3.1
4,00XX_E_MPCP_US,2012-05-01,4.6


In [20]:
df_result.account_name.nunique()

50

In [21]:
len(df_result)

1950

## Store Data

Let's store the pre-processed data as a new CSV file

In [22]:
df_result.to_csv("../data/processed/final_revenue_data.csv", index=False)